# RDF Conversions

Note: commented lines are not implemented on the Forge

In [2]:
from kgforge.core import KnowledgeGraphForge

In [3]:
forge = KnowledgeGraphForge("../../configurations/demo-forge.yml")

# Imports

In [4]:
from kgforge.core import Resource

## Resource to triples

In [5]:
jane = Resource(type="Person", name="Jane Doe")

In [6]:
forge.register(jane)

<action> _register_one
<succeeded> True


In [7]:
# forge.as_triples(jane)

In [8]:
# forge.as_triples(jane, store_metadata=True)

## Triples to Resource

In [9]:
jane = Resource(type="Person", name="Jane Doe")

In [10]:
# data = <...>

In [11]:
# resource = forge.from_triples(data)

In [12]:
# resource == jane